In [1]:
# This is in case you update modules while working
%load_ext autoreload 
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Data Check

In [3]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(
    [{'host': 'localhost', 'port': 9200}], timeout=30, max_retries=10, retry_on_timeout=True
)

In [26]:
journals = []
publication = 'arxiv'
x = 0
for hit in helpers.scan(es, index = "smartpub", query = {"query":{"match":{"journal":{"query" : publication}}}}, size = 5000):
    try:
        journals.append(hit['_source']['journal'])
    except KeyError:
        x = x + 1
        pass

In [27]:
print(len(journals), x)
print(set(journals))

39261 0
{'arxiv'}


In [6]:
res = es.search(index = "smartpub", body = {"query": {"match": {"title" : "Semantic Regularisation"}}}, size = 20)
print("Got %d Hits:" % res['hits']['total'])

for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'], hit['_source']['year'])
    print(len(hit['_source']['content'].split(' ')), 'words')

Got 327 Hits:
arxiv_1611.05490 Semantic Regularisation for Recurrent Image Annotation 2016
6673 words
arxiv_1610.07353 Filter-based regularisation for impulse response modelling 2016
6773 words
arxiv_1502.01400 Fast unsupervised Bayesian image segmentation with adaptive spatial
  regularisation 2016
6335 words
arxiv_1407.2334 The jump set under geometric regularisation. Part 2: Higher-order
  approaches 2014
13764 words
arxiv_1508.07243 Bilevel parameter learning for higher-order total variation
  regularisation models 2015
10849 words
arxiv_1310.0322 Optical Flow on Evolving Surfaces with Space and Time Regularisation 2014
7948 words
arxiv_1605.04238 Semantic Spaces 2016
12035 words
arxiv_1607.06283 Real-Time Intensity-Image Reconstruction for Event Cameras Using
  Manifold Regularisation 2016
4596 words
arxiv_1407.1531 The jump set under geometric regularisation. Part 1: Basic technique and
  first-order denoising 2015
14972 words
arxiv_1608.02565 Semantic Code Browsing 2016
7972 wor

# Labelling

In [7]:
import re
import string
import sys
import nltk
import os

from config import ROOTPATH, STANFORD_NER_PATH
from nltk.tag.stanford import StanfordNERTagger
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [8]:
model_name = 'trained_ner_DATA'
path_to_model = ROOTPATH + '/crf_trained_files/' + model_name + '.ser.gz'
ner_tagger = StanfordNERTagger(path_to_model, STANFORD_NER_PATH)
tag = 'DATA'

/usr/local/lib/python3.5/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [9]:
input_text = hit['_source']['content']
sentences = nltk.sent_tokenize(input_text)

In [12]:
tagged_sentences = []

for sentence in sentences[:10]:
    sentence = re.sub(r'\[[^\(]*?\]', r'', sentence)
    sentence = sentence.replace("@ BULLET", "")
    sentence = sentence.replace("@BULLET", "")
    sentence = sentence.replace(", ", " , ")
    sentence = sentence.replace('(', '')
    sentence = sentence.replace(')', '')
    sentence = sentence.replace('[', '')
    sentence = sentence.replace(']', '')
    sentence = sentence.replace(',', ' ,')
    sentence = sentence.replace('?', ' ?')
    sentence = sentence.replace('..', '.')
    sentence = re.sub(r"(\.)([A-Z])", r"\1 \2", sentence)
    print(sentence)
    
    tagged = ner_tagger.tag(sentence.split())
    tagged_sentences.append(tagged)

Object Boundary Guided Semantic Segmentation Qin Huang  , Chunyang Xia  , Wenchao Zheng  , Yuhang Song  , Hao Xu  , C.-C. Jay Kuo qinhuang@usc.edu Abstract.
Semantic segmentation is critical to image content under- standing and object localization.
Recent development in fully-convolutional neural network FCN has enabled accurate pixel-level labeling.
One is- sue in previous works is that the FCN based method does not exploit the object boundary information to delineate segmentation details since the object boundary label is ignored in the network training.
To tackle this problem  , we introduce a double branch fully convolutional neural network  , which separates the learning of the desirable semantic class la- beling with mask-level object proposals guided by relabeled boundaries.
This network  , called object boundary guided FCN OBG-FCN  , is able to integrate the distinct properties of object shape and class features elegantly in a fully convolutional way with a designed masking arc

In [17]:
result = []


for jj, (a, b) in enumerate(tagged_sentences[6]):
#             tag = model_name.upper()
    if b == tag:
        a = a.translate(str.maketrans('', '', string.punctuation))
        try:
            if sentences[jj + 1][1] == tag:
                temp = sentences[jj + 1][0].translate(str.maketrans('', '', string.punctuation))
                bigram = a + ' ' + temp
                result.append(bigram)
        except:
            result.append(a)
            continue
        result.append(a)
print('.', end='')

.

In [14]:
result = ['PASCAL', 'ImageNet', 'CNNbased', 'OBGMask', 'boundary', 'public', 'SPPnet', 'VOC2012', 'DeepLab', 'object', 'background', 'DTEdgeNet', 'network']

[word for word in results if not wordnet.synsets(word)]

['ImageNet',
 'CNNbased',
 'OBGMask',
 'SPPnet',
 'VOC2012',
 'DeepLab',
 'DTEdgeNet']

In [13]:
print(result)

['PASCAL', 'ImageNet', 'CNNbased', 'OBGMask', 'boundary', 'public', 'SPPnet', 'VOC2012', 'DeepLab', 'object', 'background', 'DTEdgeNet', 'network']


In [10]:
from m2_labelling import ner_labelling

In [20]:
result = ner_labelling.long_tail_labelling('MET', hit['_source']['content'])

started extraction for the MET model


/usr/local/lib/python3.5/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


...................................................................................................................................................................................................................................................................................Total of 41 filtered entities added
Object Boundary Guided Semantic Segmentation Qin Huang, Chunyang Xia, Wenchao Zheng, Yuhang Song, Hao Xu, C.-C. Jay Kuo (qinhuang@usc.edu) Abstract.
Semantic segmentation is critical to image content under- standing and object localization.
Recent development in fully-convolutional neural network (FCN) has enabled accurate pixel-level labeling.
One is- sue in previous works is that the FCN based method does not exploit the object boundary information to delineate segmentation details since the object boundary label is ignored in the network training.
To tackle this problem, we introduce a double branch fully convolutional neural network, which separates the learning of the desirab

In [22]:
[word for word in result if not wordnet.synsets(word)]

['FCNs',
 'RCNN',
 'Caffe',
 'ReLU',
 'although',
 'OBGMark',
 'Although',
 'MNCs',
 'CNNs',
 'OBPFCN',
 'ImageNet',
 'OBPFCN8s',
 'SPPnet',
 'OBGMask',
 'OBGFCN']

# Something